# 챕터 8: Skills 파이프라인 연계로 자동화

## 8-1. Skills 파이프라인 개요

여러 Skill을 순차적 또는 병렬로 연결하여 복잡한 작업을 자동화한다.

**파이프라인의 장점:**
- 복잡한 작업을 작은 단위로 분해한다
- 각 단계를 재사용 가능하게 만든다
- 중간 결과를 검증하고 디버깅할 수 있다
- 병렬 처리로 성능을 향상시킬 수 있다

**파이프라인 패턴:**
1. **순차 파이프라인**: A → B → C
2. **병렬 파이프라인**: A → (B, C, D) → E
3. **조건부 파이프라인**: A → (조건) → B 또는 C
4. **반복 파이프라인**: A → B → (조건) → A

In [ ]:
# 필요한 라이브러리 설치
%pip install anthropic python-dotenv pyyaml -q

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
MODEL = "claude-sonnet-4-5"

In [2]:
import anthropic
import os
from pathlib import Path
import yaml
import json
from typing import Dict, List, Optional, Callable, Any, Union
from dataclasses import dataclass, field
from enum import Enum
import time
from datetime import datetime

# Anthropic 클라이언트 초기화
client = anthropic.Anthropic()

print("환경 설정 완료")

환경 설정 완료


## 8-2. 파이프라인 단계 정의

각 파이프라인 단계를 나타내는 클래스를 정의한다.

In [3]:
@dataclass
class PipelineContext:
    """
    파이프라인 실행 중 데이터를 전달하는 컨텍스트
    """
    data: Dict[str, Any] = field(default_factory=dict)  # 파이프라인 데이터
    metadata: Dict[str, Any] = field(default_factory=dict)  # 메타데이터
    
    def set(self, key: str, value: Any) -> None:
        """데이터를 설정한다"""
        self.data[key] = value
    
    def get(self, key: str, default: Any = None) -> Any:
        """데이터를 가져온다"""
        return self.data.get(key, default)
    
    def set_metadata(self, key: str, value: Any) -> None:
        """메타데이터를 설정한다"""
        self.metadata[key] = value
    
    def get_metadata(self, key: str, default: Any = None) -> Any:
        """메타데이터를 가져온다"""
        return self.metadata.get(key, default)

@dataclass
class PipelineStep:
    """
    파이프라인의 단일 단계
    """
    name: str
    skill_name: str
    input_keys: List[str]  # 입력으로 사용할 컨텍스트 키
    output_key: str  # 출력을 저장할 컨텍스트 키
    prompt_template: str  # 프롬프트 템플릿
    description: Optional[str] = None
    
    def create_prompt(self, context: PipelineContext) -> str:
        """
        컨텍스트 데이터로 프롬프트를 생성한다
        
        Args:
            context: 파이프라인 컨텍스트
            
        Returns:
            생성된 프롬프트
        """
        # 템플릿에 컨텍스트 데이터를 삽입한다
        prompt_data = {key: context.get(key, '') for key in self.input_keys}
        return self.prompt_template.format(**prompt_data)

print("파이프라인 기본 클래스 정의 완료")

파이프라인 기본 클래스 정의 완료


## 8-3. 파이프라인 실행 엔진

여러 단계를 순차적으로 실행하는 파이프라인 엔진을 구현한다.

In [6]:
class SkillPipeline:
    """
    여러 Skill을 연결하여 실행하는 파이프라인
    """
    
    def __init__(self, api_key: Optional[str] = None):
        """
        파이프라인을 초기화한다
        
        Args:
            api_key: Anthropic API 키
        """
        self.client = anthropic.Anthropic()
        self.skills: Dict[str, Dict] = {}  # 로드된 Skills
        self.steps: List[PipelineStep] = []  # 파이프라인 단계들
        self.execution_log: List[Dict] = []  # 실행 로그
    
    def load_skill(self, skill_path: str) -> None:
        """
        Skill 파일을 로드한다
        
        Args:
            skill_path: Skill 파일 경로
        """
        path = Path(skill_path)
        
        with open(path, 'r', encoding='utf-8') as f:
            if path.suffix == '.yaml':
                skill_data = yaml.safe_load(f)
            else:
                raise ValueError(f"지원하지 않는 파일 형식: {path.suffix}")
        
        skill_name = skill_data['name']
        self.skills[skill_name] = skill_data
        print(f"Skill 로드: {skill_name}")
    
    def add_step(self, step: PipelineStep) -> None:
        """
        파이프라인에 단계를 추가한다
        
        Args:
            step: 추가할 파이프라인 단계
        """
        if step.skill_name not in self.skills:
            raise ValueError(f"Skill '{step.skill_name}'이 로드되지 않았다")
        
        self.steps.append(step)
        print(f"단계 추가: {step.name} (Skill: {step.skill_name})")
    
    def execute_step(
        self,
        step: PipelineStep,
        context: PipelineContext
    ) -> str:
        """
        단일 파이프라인 단계를 실행한다
        
        Args:
            step: 실행할 단계
            context: 파이프라인 컨텍스트
            
        Returns:
            실행 결과
        """
        skill = self.skills[step.skill_name]
        config = skill.get('config', {})
        
        # 프롬프트 생성
        user_prompt = step.create_prompt(context)
        
        # API 호출
        start_time = time.time()
        
        response = self.client.messages.create(
            model=config.get('model', MODEL),
            max_tokens=config.get('max_tokens', 4000),
            temperature=config.get('temperature', 0.7),
            system=skill['content'],
            messages=[
                {
                    "role": "user",
                    "content": user_prompt
                }
            ]
        )
        
        elapsed_time = time.time() - start_time
        result = response.content[0].text
        
        # 실행 로그 기록
        self.execution_log.append({
            'step_name': step.name,
            'skill_name': step.skill_name,
            'timestamp': datetime.now().isoformat(),
            'elapsed_time': elapsed_time,
            'input_prompt': user_prompt[:200] + '...' if len(user_prompt) > 200 else user_prompt,
            'output_length': len(result)
        })
        
        return result
    
    def run(self, initial_data: Dict[str, Any]) -> PipelineContext:
        """
        파이프라인을 실행한다
        
        Args:
            initial_data: 초기 입력 데이터
            
        Returns:
            실행 완료된 컨텍스트
        """
        context = PipelineContext(data=initial_data.copy())
        context.set_metadata('start_time', datetime.now().isoformat())
        
        print("\n" + "=" * 80)
        print("파이프라인 실행 시작")
        print("=" * 80)
        
        for i, step in enumerate(self.steps, 1):
            print(f"\n[{i}/{len(self.steps)}] {step.name} 실행 중...")
            
            try:
                result = self.execute_step(step, context)
                context.set(step.output_key, result)
                print(f"  완료 (출력 길이: {len(result)} 문자)")
            except Exception as e:
                print(f"  오류 발생: {e}")
                context.set_metadata(f'error_{step.name}', str(e))
                raise
        
        context.set_metadata('end_time', datetime.now().isoformat())
        
        print("\n" + "=" * 80)
        print("파이프라인 실행 완료")
        print("=" * 80)
        
        return context
    
    def get_execution_summary(self) -> str:
        """
        실행 로그 요약을 반환한다
        
        Returns:
            실행 요약 문자열
        """
        if not self.execution_log:
            return "실행 로그가 없다"
        
        summary = "\n파이프라인 실행 요약:\n"
        summary += "=" * 80 + "\n"
        
        total_time = sum(log['elapsed_time'] for log in self.execution_log)
        
        for log in self.execution_log:
            summary += f"\n단계: {log['step_name']}\n"
            summary += f"  Skill: {log['skill_name']}\n"
            summary += f"  실행 시간: {log['elapsed_time']:.2f}초\n"
            summary += f"  출력 길이: {log['output_length']} 문자\n"
        
        summary += f"\n총 실행 시간: {total_time:.2f}초\n"
        
        return summary

print("SkillPipeline 클래스 정의 완료")

SkillPipeline 클래스 정의 완료


## 8-4. 실습용 Skills 생성

파이프라인 예제를 위한 여러 Skill을 생성한다.

In [7]:
# skills 디렉토리 생성
skills_dir = Path("skills/pipeline")
skills_dir.mkdir(exist_ok=True)

# 1. 아이디어 생성 Skill
idea_generator_skill = {
    'name': 'idea_generator',
    'version': '1.0.0',
    'description': '창의적인 아이디어 생성 전문가',
    'content': """# 아이디어 생성 전문가 Skill

## 역할
주어진 주제나 문제에 대해 창의적이고 실용적인 아이디어를 생성하는 전문가다.

## 아이디어 생성 원칙
1. 다양성: 다양한 관점에서 아이디어를 제시한다
2. 구체성: 추상적이지 않고 구체적인 아이디어를 제공한다
3. 실현 가능성: 실제로 실행 가능한 아이디어를 우선한다
4. 혁신성: 기존과 차별화된 새로운 접근을 시도한다

## 출력 형식
- 3-5개의 아이디어를 제시한다
- 각 아이디어는 제목과 간단한 설명을 포함한다
- 구현 난이도를 표시한다

## 스타일
- 문장은 ~다 체를 사용한다
- 열정적이고 긍정적인 톤을 유지한다
""",
    'config': {
        'max_tokens': 2000,
        'temperature': 0.8
    }
}

# 2. 기획서 작성 Skill
proposal_writer_skill = {
    'name': 'proposal_writer',
    'version': '1.0.0',
    'description': '프로젝트 기획서 작성 전문가',
    'content': """# 기획서 작성 전문가 Skill

## 역할
아이디어를 체계적인 프로젝트 기획서로 작성하는 전문가다.

## 기획서 구성
1. 개요: 프로젝트 목적과 배경
2. 목표: 달성하고자 하는 구체적 목표
3. 범위: 프로젝트 범위와 제약사항
4. 방법론: 실행 방법과 접근 전략
5. 일정: 단계별 타임라인
6. 리소스: 필요한 인력과 자원
7. 리스크: 예상 리스크와 대응 방안

## 작성 원칙
- 논리적이고 체계적으로 구성한다
- 정량적 지표를 포함한다
- 실행 가능성을 강조한다

## 스타일
- 문장은 ~다 체를 사용한다
- 전문적이고 명확한 언어를 사용한다
""",
    'config': {
        'max_tokens': 3000,
        'temperature': 0.5
    }
}

# 3. 마케팅 전략 Skill
marketing_strategy_skill = {
    'name': 'marketing_strategist',
    'version': '1.0.0',
    'description': '마케팅 전략 수립 전문가',
    'content': """# 마케팅 전략가 Skill

## 역할
제품이나 서비스의 마케팅 전략을 수립하는 전문가다.

## 전략 수립 프레임워크
1. 시장 분석: 타겟 시장과 경쟁 환경
2. 포지셔닝: 차별화 포인트와 가치 제안
3. 채널 전략: 효과적인 마케팅 채널 선정
4. 콘텐츠 전략: 메시지와 크리에이티브 방향
5. 실행 계획: 구체적인 캠페인 플랜
6. 성과 측정: KPI와 측정 방법

## 작성 원칙
- 데이터 기반 의사결정을 강조한다
- 구체적인 액션 아이템을 제시한다
- ROI를 고려한 전략을 수립한다

## 스타일
- 문장은 ~다 체를 사용한다
- 설득력 있고 전략적인 언어를 사용한다
""",
    'config': {
        'max_tokens': 3000,
        'temperature': 0.6
    }
}

# 4. 요약 전문가 Skill
summarizer_skill = {
    'name': 'summarizer',
    'version': '1.0.0',
    'description': '문서 요약 전문가',
    'content': """# 요약 전문가 Skill

## 역할
긴 문서를 핵심만 추려 간결하게 요약하는 전문가다.

## 요약 원칙
1. 핵심 보존: 가장 중요한 정보를 우선한다
2. 간결성: 불필요한 세부사항을 제거한다
3. 구조화: 논리적 흐름을 유지한다
4. 명확성: 이해하기 쉬운 언어를 사용한다

## 출력 형식
- 한 문단으로 전체 요약을 제공한다
- 3-5개의 핵심 포인트를 나열한다

## 스타일
- 문장은 ~다 체를 사용한다
- 객관적이고 중립적인 톤을 유지한다
""",
    'config': {
        'max_tokens': 1500,
        'temperature': 0.3
    }
}

# Skills 저장
skills_to_save = [
    idea_generator_skill,
    proposal_writer_skill,
    marketing_strategy_skill,
    summarizer_skill
]

for skill in skills_to_save:
    skill_path = skills_dir / f"{skill['name']}.yaml"
    with open(skill_path, 'w', encoding='utf-8') as f:
        yaml.dump(skill, f, allow_unicode=True, default_flow_style=False)
    print(f"Skill 저장: {skill_path}")

Skill 저장: skills/pipeline/idea_generator.yaml
Skill 저장: skills/pipeline/proposal_writer.yaml
Skill 저장: skills/pipeline/marketing_strategist.yaml
Skill 저장: skills/pipeline/summarizer.yaml


## 8-5. 예제 1: 제품 개발 파이프라인

아이디어 생성 → 기획서 작성 → 마케팅 전략 → 요약 순서로 실행되는 파이프라인이다.

In [8]:
# 파이프라인 생성
pipeline = SkillPipeline()

# Skills 로드
pipeline.load_skill('skills/pipeline/idea_generator.yaml')
pipeline.load_skill('skills/pipeline/proposal_writer.yaml')
pipeline.load_skill('skills/pipeline/marketing_strategist.yaml')
pipeline.load_skill('skills/pipeline/summarizer.yaml')

print("\n파이프라인 Skills 로드 완료")

Skill 로드: idea_generator
Skill 로드: proposal_writer
Skill 로드: marketing_strategist
Skill 로드: summarizer

파이프라인 Skills 로드 완료


In [9]:
# 파이프라인 단계 정의

# 1단계: 아이디어 생성
step1 = PipelineStep(
    name="아이디어 생성",
    skill_name="idea_generator",
    input_keys=["topic"],
    output_key="ideas",
    prompt_template="""{topic}에 대한 혁신적인 제품 아이디어를 3개 제안해줘.
각 아이디어는 타겟 고객과 핵심 가치를 포함해야 한다."""
)

# 2단계: 기획서 작성
step2 = PipelineStep(
    name="기획서 작성",
    skill_name="proposal_writer",
    input_keys=["ideas"],
    output_key="proposal",
    prompt_template="""다음 아이디어 중 첫 번째 아이디어를 선택하여 상세한 프로젝트 기획서를 작성해줘:

{ideas}

기획서에는 개요, 목표, 범위, 방법론, 일정, 리소스, 리스크를 모두 포함해야 한다."""
)

# 3단계: 마케팅 전략 수립
step3 = PipelineStep(
    name="마케팅 전략 수립",
    skill_name="marketing_strategist",
    input_keys=["proposal"],
    output_key="marketing_strategy",
    prompt_template="""다음 프로젝트 기획서를 바탕으로 종합적인 마케팅 전략을 수립해줘:

{proposal}

시장 분석, 포지셔닝, 채널 전략, 콘텐츠 전략, 실행 계획, 성과 측정을 모두 포함해야 한다."""
)

# 4단계: 전체 요약
step4 = PipelineStep(
    name="전체 요약",
    skill_name="summarizer",
    input_keys=["ideas", "proposal", "marketing_strategy"],
    output_key="summary",
    prompt_template="""다음 내용들을 종합하여 경영진에게 보고할 수 있는 간결한 요약본을 작성해줘:

아이디어:
{ideas}

기획서:
{proposal}

마케팅 전략:
{marketing_strategy}

핵심만 추려서 요약해줘."""
)

# 파이프라인에 단계 추가
pipeline.add_step(step1)
pipeline.add_step(step2)
pipeline.add_step(step3)
pipeline.add_step(step4)

print("\n파이프라인 단계 구성 완료")

단계 추가: 아이디어 생성 (Skill: idea_generator)
단계 추가: 기획서 작성 (Skill: proposal_writer)
단계 추가: 마케팅 전략 수립 (Skill: marketing_strategist)
단계 추가: 전체 요약 (Skill: summarizer)

파이프라인 단계 구성 완료


In [10]:
# 파이프라인 실행
initial_data = {
    "topic": "원격 근무자를 위한 생산성 향상 도구"
}

result_context = pipeline.run(initial_data)

# 실행 요약 출력
print("\n" + pipeline.get_execution_summary())


파이프라인 실행 시작

[1/4] 아이디어 생성 실행 중...
  완료 (출력 길이: 1340 문자)

[2/4] 기획서 작성 실행 중...
  완료 (출력 길이: 3727 문자)

[3/4] 마케팅 전략 수립 실행 중...
  완료 (출력 길이: 3359 문자)

[4/4] 전체 요약 실행 중...
  완료 (출력 길이: 838 문자)

파이프라인 실행 완료


파이프라인 실행 요약:

단계: 아이디어 생성
  Skill: idea_generator
  실행 시간: 24.93초
  출력 길이: 1340 문자

단계: 기획서 작성
  Skill: proposal_writer
  실행 시간: 51.11초
  출력 길이: 3727 문자

단계: 마케팅 전략 수립
  Skill: marketing_strategist
  실행 시간: 51.78초
  출력 길이: 3359 문자

단계: 전체 요약
  Skill: summarizer
  실행 시간: 15.64초
  출력 길이: 838 문자

총 실행 시간: 143.47초



In [11]:
# 각 단계의 결과 확인
print("\n" + "=" * 80)
print("단계별 결과 미리보기")
print("=" * 80)

print("\n[1단계: 아이디어 생성]")
print(result_context.get('ideas')[:500] + "...")

print("\n[2단계: 기획서 작성]")
print(result_context.get('proposal')[:500] + "...")

print("\n[3단계: 마케팅 전략]")
print(result_context.get('marketing_strategy')[:500] + "...")

print("\n[4단계: 전체 요약]")
print(result_context.get('summary'))


단계별 결과 미리보기

[1단계: 아이디어 생성]
# 원격 근무자를 위한 혁신적인 생산성 도구 아이디어

## 1. 스마트 컨텍스트 스위처 "FlowShift"

**핵심 개념:** AI 기반 업무 컨텍스트 자동 전환 시스템

**타겟 고객:** 
- 여러 프로젝트를 동시에 진행하는 프리랜서
- 멀티태스킹이 잦은 스타트업 직원
- 클라이언트 미팅이 많은 컨설턴트

**핵심 가치:**
- 업무 전환 시 발생하는 컨텍스트 스위칭 비용을 80% 감소시킨다
- 캘린더 일정과 연동하여 자동으로 작업 환경을 세팅한다
- 각 프로젝트별 필요한 앱, 파일, 브라우저 탭을 원클릭으로 불러온다
- 업무 전환 전 5분간의 "마무리 타임"을 제공하여 이전 작업을 정리할 수 있게 한다

**구현 난이도:** ⭐⭐⭐☆☆ (중상)

---

## 2. 감정 기반 근무 최적화 플랫폼 "MoodWork"

**핵심 개념:** 생체 리듬과 감정 상태를 분석하여 최적의 업무 스케줄을 제안하는 웰니스 생산성 도구

**타겟 고객:**
- 번아웃 위험이 높은 원격 근무...

[2단계: 기획서 작성]
# FlowShift 프로젝트 기획서

## 1. 개요

### 1.1 프로젝트 배경
현대 원격 근무 환경에서 지식 근로자들은 평균적으로 하루 10회 이상 업무 컨텍스트를 전환한다. 캘리포니아 대학교 어바인 캠퍼스의 연구에 따르면, 업무 전환 후 이전 작업의 집중도를 회복하는 데 평균 23분 15초가 소요된다. 이는 하루 약 3.8시간의 생산성 손실로 이어지며, 연간 기준으로 근로자 1인당 약 450시간의 비효율을 발생시킨다.

특히 프리랜서, 스타트업 직원, 컨설턴트 등 멀티태스킹이 필수적인 직군에서 이 문제는 더욱 심각하다. 이들은 프로젝트별로 상이한 도구, 파일, 커뮤니케이션 채널을 사용하며, 업무 전환 시마다 필요한 리소스를 수동으로 찾고 열어야 하는 번거로움을 겪는다.

FlowShift는 AI 기반 자동화를 통해 이러한 컨텍스트 스위칭 비용을 최소화하고, 원격 근무자의 생산성을 극대화하는

## 8-6. 결과 저장 및 관리

In [12]:
# 결과를 파일로 저장하는 함수
def save_pipeline_results(
    context: PipelineContext,
    output_dir: str = "pipeline_output"
) -> None:
    """
    파이프라인 결과를 파일로 저장한다
    
    Args:
        context: 파이프라인 컨텍스트
        output_dir: 출력 디렉토리
    """
    output_path = Path(output_dir)
    output_path.mkdir(exist_ok=True)
    
    # 타임스탬프로 폴더 생성
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    run_dir = output_path / f"run_{timestamp}"
    run_dir.mkdir(exist_ok=True)
    
    # 각 단계의 결과를 개별 파일로 저장
    for key, value in context.data.items():
        if isinstance(value, str):
            file_path = run_dir / f"{key}.txt"
            with open(file_path, 'w', encoding='utf-8') as f:
                f.write(value)
            print(f"저장: {file_path}")
    
    # 메타데이터 저장
    metadata_path = run_dir / "metadata.json"
    with open(metadata_path, 'w', encoding='utf-8') as f:
        json.dump(context.metadata, f, ensure_ascii=False, indent=2)
    print(f"메타데이터 저장: {metadata_path}")
    
    # 전체 결과를 하나의 마크다운 파일로 저장
    markdown_path = run_dir / "full_report.md"
    with open(markdown_path, 'w', encoding='utf-8') as f:
        f.write(f"# 파이프라인 실행 결과\n\n")
        f.write(f"실행 시간: {context.get_metadata('start_time')}\n\n")
        
        for key, value in context.data.items():
            if isinstance(value, str):
                f.write(f"## {key.replace('_', ' ').title()}\n\n")
                f.write(value)
                f.write("\n\n---\n\n")
    
    print(f"전체 보고서 저장: {markdown_path}")
    print(f"\n모든 결과가 {run_dir}에 저장되었다")

# 결과 저장
save_pipeline_results(result_context)

저장: pipeline_output/run_20251108_160720/topic.txt
저장: pipeline_output/run_20251108_160720/ideas.txt
저장: pipeline_output/run_20251108_160720/proposal.txt
저장: pipeline_output/run_20251108_160720/marketing_strategy.txt
저장: pipeline_output/run_20251108_160720/summary.txt
메타데이터 저장: pipeline_output/run_20251108_160720/metadata.json
전체 보고서 저장: pipeline_output/run_20251108_160720/full_report.md

모든 결과가 pipeline_output/run_20251108_160720에 저장되었다


## 8-7. 예제 2: 조건부 파이프라인

중간 결과에 따라 다음 단계를 선택하는 조건부 파이프라인이다.

In [13]:
class ConditionalPipeline(SkillPipeline):
    """
    조건부 분기를 지원하는 파이프라인
    """
    
    def add_conditional_step(
        self,
        condition_fn: Callable[[PipelineContext], bool],
        true_step: PipelineStep,
        false_step: PipelineStep
    ) -> None:
        """
        조건부 단계를 추가한다
        
        Args:
            condition_fn: 조건 판단 함수
            true_step: 조건이 참일 때 실행할 단계
            false_step: 조건이 거짓일 때 실행할 단계
        """
        self.steps.append({
            'type': 'conditional',
            'condition_fn': condition_fn,
            'true_step': true_step,
            'false_step': false_step
        })
    
    def run(self, initial_data: Dict[str, Any]) -> PipelineContext:
        """
        조건부 로직을 포함한 파이프라인을 실행한다
        """
        context = PipelineContext(data=initial_data.copy())
        context.set_metadata('start_time', datetime.now().isoformat())
        
        print("\n" + "=" * 80)
        print("조건부 파이프라인 실행 시작")
        print("=" * 80)
        
        for i, step in enumerate(self.steps, 1):
            if isinstance(step, dict) and step.get('type') == 'conditional':
                # 조건부 단계
                condition_result = step['condition_fn'](context)
                selected_step = step['true_step'] if condition_result else step['false_step']
                
                print(f"\n[{i}/{len(self.steps)}] 조건 평가: {'참' if condition_result else '거짓'}")
                print(f"  선택된 단계: {selected_step.name}")
                
                result = self.execute_step(selected_step, context)
                context.set(selected_step.output_key, result)
            else:
                # 일반 단계
                print(f"\n[{i}/{len(self.steps)}] {step.name} 실행 중...")
                result = self.execute_step(step, context)
                context.set(step.output_key, result)
        
        context.set_metadata('end_time', datetime.now().isoformat())
        print("\n" + "=" * 80)
        print("파이프라인 실행 완료")
        print("=" * 80)
        
        return context

print("ConditionalPipeline 클래스 정의 완료")

ConditionalPipeline 클래스 정의 완료


## 8-8. 파이프라인 템플릿 저장 및 재사용

In [14]:
class PipelineTemplate:
    """
    재사용 가능한 파이프라인 템플릿
    """
    
    @staticmethod
    def save_template(
        pipeline: SkillPipeline,
        template_name: str,
        output_dir: str = "pipeline_templates"
    ) -> None:
        """
        파이프라인을 템플릿으로 저장한다
        
        Args:
            pipeline: 저장할 파이프라인
            template_name: 템플릿 이름
            output_dir: 출력 디렉토리
        """
        output_path = Path(output_dir)
        output_path.mkdir(exist_ok=True)
        
        # 파이프라인 구조를 딕셔너리로 변환
        template_data = {
            'name': template_name,
            'created_at': datetime.now().isoformat(),
            'skills': list(pipeline.skills.keys()),
            'steps': [
                {
                    'name': step.name,
                    'skill_name': step.skill_name,
                    'input_keys': step.input_keys,
                    'output_key': step.output_key,
                    'prompt_template': step.prompt_template,
                    'description': step.description
                }
                for step in pipeline.steps
            ]
        }
        
        template_path = output_path / f"{template_name}.json"
        with open(template_path, 'w', encoding='utf-8') as f:
            json.dump(template_data, f, ensure_ascii=False, indent=2)
        
        print(f"템플릿 저장: {template_path}")
    
    @staticmethod
    def load_template(
        template_path: str,
        api_key: Optional[str] = None
    ) -> SkillPipeline:
        """
        템플릿에서 파이프라인을 로드한다
        
        Args:
            template_path: 템플릿 파일 경로
            api_key: API 키
            
        Returns:
            로드된 파이프라인
        """
        with open(template_path, 'r', encoding='utf-8') as f:
            template_data = json.load(f)
        
        # 파이프라인 재구성
        pipeline = SkillPipeline(api_key=api_key)
        
        # Skills 로드
        for skill_name in template_data['skills']:
            skill_path = f"skills/{skill_name}.yaml"
            if Path(skill_path).exists():
                pipeline.load_skill(skill_path)
        
        # 단계 재구성
        for step_data in template_data['steps']:
            step = PipelineStep(
                name=step_data['name'],
                skill_name=step_data['skill_name'],
                input_keys=step_data['input_keys'],
                output_key=step_data['output_key'],
                prompt_template=step_data['prompt_template'],
                description=step_data.get('description')
            )
            pipeline.add_step(step)
        
        print(f"템플릿 로드 완료: {template_data['name']}")
        return pipeline

# 현재 파이프라인을 템플릿으로 저장
PipelineTemplate.save_template(pipeline, "product_development_pipeline")

print("\n파이프라인 템플릿 시스템 준비 완료")

템플릿 저장: pipeline_templates/product_development_pipeline.json

파이프라인 템플릿 시스템 준비 완료


## 8-9. 생성된 파일 확인

In [15]:
print("\n" + "=" * 80)
print("생성된 파일 목록")
print("=" * 80)

# Skills 목록
print("\n[Skills]")
for file_path in sorted(skills_dir.glob("*.yaml")):
    size = file_path.stat().st_size
    print(f"  {file_path.name} ({size:,} bytes)")

# 파이프라인 출력 결과
pipeline_output_dir = Path("pipeline_output")
if pipeline_output_dir.exists():
    print("\n[파이프라인 실행 결과]")
    for run_dir in sorted(pipeline_output_dir.glob("run_*")):
        print(f"  {run_dir.name}/")
        for file_path in sorted(run_dir.glob("*")):
            size = file_path.stat().st_size
            print(f"    {file_path.name} ({size:,} bytes)")

# 템플릿
template_dir = Path("pipeline_templates")
if template_dir.exists():
    print("\n[파이프라인 템플릿]")
    for file_path in sorted(template_dir.glob("*.json")):
        size = file_path.stat().st_size
        print(f"  {file_path.name} ({size:,} bytes)")


생성된 파일 목록

[Skills]
  idea_generator.yaml (937 bytes)
  marketing_strategist.yaml (911 bytes)
  proposal_writer.yaml (895 bytes)
  summarizer.yaml (726 bytes)

[파이프라인 실행 결과]
  run_20251108_160720/
    full_report.md (17,727 bytes)
    ideas.txt (2,904 bytes)
    marketing_strategy.txt (6,211 bytes)
    metadata.json (92 bytes)
    proposal.txt (6,683 bytes)
    summary.txt (1,700 bytes)
    topic.txt (50 bytes)

[파이프라인 템플릿]
  product_development_pipeline.json (2,016 bytes)
